# Achate a curva
> Número de casos ativos e estimativa de leitos de UTI total e para pacientes covid-19

- toc: true
- badges: true
- comments: true
- author: Cleber Jorge Amaral
- categories: [jupyter]
- image: images/brazil-flatten-the-curve.png

In [1]:
#hide
import pandas as pd
import altair as alt
import math
from altair_saver import save
from IPython.display import HTML

CHART_WIDTH = 600
CHART_HEIGHT = 400

/usr/local/lib/python3.8/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [2]:
#hide
STATE_COLUMN = "Estado"
DATE_COLUMN = "Data"
CONFIRMED_CASES = 'Casos confirmados'
VALUE = "Valor"
CASES_NEED_ICU = 0.05
POPULATION = 211000000
ICU_PER_100k = 20
TOTAL_ICU = 33905
AVAILABLE_PERCENTAGE_ICU = 0.35
TOTAL_AVAILABLE_ICU = TOTAL_ICU * AVAILABLE_PERCENTAGE_ICU
CASES_NEED_ICU_COLUMN = "Precisam de UTI "+"{:.0f}".format(CASES_NEED_ICU*100)+"% dos casos"
PATIENTS = "Pacientes"
UPPER_BOUND_ICU_NEED_STR = "Limite superior de necessidade de UTI"
LOWER_BOUND_ICU_NEED_STR = "Limite inferior de necessidade de UTI"
UPPER_BOUND_ICU_NEED = 0.12
LOWER_BOUND_ICU_NEED = 0.01

In [3]:
url = ('https://data.brasil.io/dataset/covid19/caso.csv.gz')
src = pd.read_csv(url)
df = src[(src['place_type'] == 'state')]
df = df.rename(columns={
     "state": STATE_COLUMN, 
     "date":DATE_COLUMN, 
     "confirmed":VALUE,
     "actives": CONFIRMED_CASES
})
df.sample(5)

,Data,Estado,city,place_type,Valor,deaths,order_for_place,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate
8590,2020-04-13,SC,NaN,state,826,26,33,False,7164788.0,42.0,11.52860,0.0315
50376,2020-04-09,MS,NaN,state,89,2,27,False,2778986.0,50.0,3.20261,0.0225
2364,2020-03-13,RS,NaN,state,4,0,4,False,11377239.0,43.0,0.03516,0.0000
42831,2020-03-24,PB,NaN,state,3,0,13,False,4018127.0,25.0,0.07466,0.0000
59424,2020-03-09,MG,NaN,state,1,0,2,False,21168791.0,31.0,0.00472,0.0000


In [4]:
#hide
data = df.copy()
df = data.copy()
df[UPPER_BOUND_ICU_NEED_STR] = round(df[VALUE] * (UPPER_BOUND_ICU_NEED),0)
df[LOWER_BOUND_ICU_NEED_STR] = round(df[VALUE] * (LOWER_BOUND_ICU_NEED),0)
df = df.rename(columns={VALUE:CONFIRMED_CASES})
df[CONFIRMED_CASES] = round(df[CONFIRMED_CASES] * (CASES_NEED_ICU),0)
df.tail()

,Data,Estado,city,place_type,Casos confirmados,deaths,order_for_place,is_last,estimated_population_2019,city_ibge_code,confirmed_per_100k_inhabitants,death_rate,Limite superior de necessidade de UTI,Limite inferior de necessidade de UTI
79209,2020-03-13,AL,NaN,state,0.0,0,5,False,3337357.0,27.0,0.02996,0.0,0.0,0.0
79210,2020-03-12,AL,NaN,state,0.0,0,4,False,3337357.0,27.0,0.02996,0.0,0.0,0.0
79211,2020-03-10,AL,NaN,state,0.0,0,3,False,3337357.0,27.0,0.02996,0.0,0.0,0.0
79212,2020-03-09,AL,NaN,state,0.0,0,2,False,3337357.0,27.0,0.02996,0.0,0.0,0.0
79213,2020-03-08,AL,NaN,state,0.0,0,1,False,3337357.0,27.0,0.02996,0.0,0.0,0.0


In [10]:
#hide_input
input_dropdown = alt.binding_select(options=df[STATE_COLUMN].unique())
selection1 = alt.selection_single(fields=[STATE_COLUMN], bind=input_dropdown, name=' ')
selection2 = alt.selection_multi(fields=[STATE_COLUMN], on='mouseover')
color = alt.condition(selection1 | selection2,
                    alt.Color(STATE_COLUMN+':N', 
                              scale=alt.Scale(scheme='tableau20', reverse=False), legend=None),
                              alt.value('#ffbf79')
                     )

chart = alt.Chart(df).mark_line().encode(
    x=alt.X(DATE_COLUMN+':O', axis=alt.Axis(title=DATE_COLUMN)),
    y=alt.Y(CONFIRMED_CASES+':Q', axis=alt.Axis(
        title=["Estimativa do percentual de casos confirmados que precisam de UTI: {:.0f}".format(CASES_NEED_ICU*100)+"%",
               UPPER_BOUND_ICU_NEED_STR+": {:.0f}".format(UPPER_BOUND_ICU_NEED*100)+"%",
               LOWER_BOUND_ICU_NEED_STR+": {:.0f}".format(LOWER_BOUND_ICU_NEED*100)+"%"])),
    color=color,
    tooltip=[
        DATE_COLUMN, 
        STATE_COLUMN, 
        CONFIRMED_CASES
    ],
    order=alt.Order(
    STATE_COLUMN,
    sort='ascending'
    )
).properties(
    title=[
        "Achate a curva - apenas os casos confirmados que estima-se que precisem de UTI",
        " * ver premissas"
    ]
).add_selection(
    selection1, selection2
).transform_filter(
    selection1
)

shades = alt.Chart(df).mark_area().encode(
    x=DATE_COLUMN+':O',
    y=LOWER_BOUND_ICU_NEED_STR+":Q",
    y2=UPPER_BOUND_ICU_NEED_STR+':Q',
    opacity = alt.condition(selection1, alt.value(0.2), alt.value(0.5))
)

x1line = alt.Chart(pd.DataFrame({'y': [TOTAL_AVAILABLE_ICU]})).mark_rule(color='#e42726', strokeWidth=2).encode(
    y='y:Q'
)
text1 = x1line.mark_text(align='left', x=5, dy=-10, color='#e42726', strokeWidth=1).encode(
    text=alt.value("UTIs para pacientes COVID19: "+"{:.0f}".format(TOTAL_AVAILABLE_ICU)+" unidades")
)
text2 = x1line.mark_text(align='left', x=5, dy=10, color='#e42726', strokeWidth=1).encode(
    text=alt.value("{:.0f}".format(AVAILABLE_PERCENTAGE_ICU*100)+"% do total de "+"{:.0f}".format(TOTAL_ICU)+" UTIs do país")   
)

legend = alt.Chart(df).mark_point().encode(
    y=alt.Y(STATE_COLUMN+':N', axis=alt.Axis(orient='right')),
    color=color
).add_selection(
    selection1, selection2
)

plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) | legend
#plot = chart.properties(width=CHART_WIDTH, height=CHART_HEIGHT) + x1line + text1 + text2 + shades | legend
plot

alt.HConcatChart(...)

In [6]:
#hide_input
print("Premissas :")
print("- O Brasil tem 33905 leitos de UTI adulto (SUS/Não SUS), dado de 06/05/2020 (fonte: painel de insumos e leitos)")
print("- {:.0f}".format(AVAILABLE_PERCENTAGE_ICU*100)+"% de UTIs disponíveis para pacientes com covid19 (fonte: artigo oglobo)")
print("- Estima-se que cerca de "+"{:.0f}".format(CASES_NEED_ICU*100)+"% dos pacientes com covid-19 venham a precisar de tratamento intensivo. No pior caso registrado, na Itália, cerca de 12% precisaram de UTI e no melhor caso, China, cerca de 1% dos infectados precisaram de UTI (fonte: artigo the lancet)")

Premissas :
- O Brasil tem 33905 leitos de UTI adulto (SUS/Não SUS), dado de 06/05/2020 (fonte: painel de insumos e leitos)
- 35% de UTIs disponíveis para pacientes com covid19 (fonte: artigo oglobo)
- Estima-se que cerca de 5% dos pacientes com covid-19 venham a precisar de tratamento intensivo. No pior caso registrado, na Itália, cerca de 12% precisaram de UTI e no melhor caso, China, cerca de 1% dos infectados precisaram de UTI (fonte: artigo the lancet)


In [7]:
#hide_input
HTML(f'<small class="float-right">Última atualização em {pd.to_datetime(LAST_DATE).strftime("%d/%m/%Y")}</small>')

NameError: name 'LAST_DATE' is not defined

Based on the work of [Alonso Silva Allende](https://covid19dashboards.com/jupyter/2020/04/27/Covid-19-Overview-Chile.html) and [Joao B. Duarte](https://github.com/github/covid19-dashboard/blob/master/_notebooks/2020-03-19-estimating_infected.ipynb), adapted by [Cleber Jorge Amaral](http://cleberjamaral.github.io/). 

Dados:
- covid19: [CSSEGISandData](https://github.com/CSSEGISandData/COVID-19)
- quantidade de leitos: [painel de insumos e leitos](https://covid-insumos.saude.gov.br/paineis/insumos/painel_leitos.php), [elpais](https://brasil.elpais.com/brasil/2020-04-28/sem-transparencia-sobre-fila-para-utis-justica-opera-para-garantir-atendimento-a-pacientes-de-covid-19.html), [elpais2](https://brasil.elpais.com/sociedade/2020-04-15/sem-leitos-de-uti-municipios-pequenos-temem-por-estrutura-limitada-para-transferir-pacientes-graves-com-a-covid-19.html), [bcc](https://www.bbc.com/portuguese/brasil-52137553) e [ministério da saúde](https://www.saude.gov.br/noticias/agencia-saude/46772-brasil-ganha-reforco-de-1-134-leitos-de-uti-no-combate-ao-coronavirus)
- leitos disponíveis: [artigo oglobo](https://oglobo.globo.com/sociedade/coronavirus/coronavirus-ministerio-estima-que-sus-tem-de-12-13-mil-leitos-de-uti-disponiveis-para-atender-pacientes-1-24328523)
- pacientes que precisam de UTI: [artigo the lancet](https://linkinghub.elsevier.com/retrieve/pii/S2213260020301612)

In [ ]:
#hide
save(plot,"../images/brazil-flatten-the-curve.png")